# Collocates

> Functionality for collocation analysis.

In [ ]:
#| default_exp collocates

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from __future__ import annotations
import numpy as np
import time
import polars as pl
from fastcore.basics import patch
import math

In [ ]:
#| export
from conc.corpus import Corpus
from conc.result import Result
from conc.core import logger, PAGE_SIZE

In [ ]:
#| export
class Collocates:
	""" Class for collocation analysis reporting. """
	def __init__(self,
			  corpus:Corpus # Corpus instance
			  ): 
		self.corpus = corpus


In [ ]:
#| hide
## Collocation methods
# 
# # @patch
# def collocates(self: Corpus, word, nice_word, constrain_to = False, context_length = 5, limit = 50, cutoff=5, stat = 'ld', output = False):
#     elements = {}
#     nodes = []
#     edges = []

#     coll_token_sequence, coll_index_id = tokenize_string(corpus_name, word)
#     token_id = coll_token_sequence[0][0]
    
#     #print(token_id, nice_word)
    
#     if constrain_to == False:
#         nodes.append((token_id, {'label': nice_word, 'size': 1}))
    
#     #print(coll_token_sequence, coll_index_id)
#     coll_token_index = profile_get_token_index(corpus_name, coll_token_sequence, coll_index_id) # get_token_positions
#     #print(coll_token_index)
#     positional_columns, concordance = profile_get_concordance(corpus_name, coll_token_sequence, coll_token_index, context_words = context_length, index_id = LOWER)
#     #print(concordance)
#     collocates = []
#     for row in concordance:
#         if eof_token in row:
#             indexes = np.where(np.array(row) == eof_token)[0]
#             #print(indexes)
#             slice_min = -1
#             slice_max = context_length * 2 + 1
#             for i in indexes:
#                 if i < context_length and i > slice_min:
#                     slice_min = i
#                 elif i > context_length and i < slice_max:
#                     #print('***')
#                     slice_max = i
#             slice_min += 1
#             #slice_max -= 1
#             #print(slice_min,slice_max)
#             #print(row)
#             #print(row[slice_min:slice_max])
#             collocates.append(row[slice_min:slice_max])
#         else:
#             collocates.append(row)

#     node_frequency = len(collocates)
            
#     if len(collocates) < 1:
#         print('no collocates')
#     else:
#         #print(collocates)
#         collocates = np.concatenate(collocates)
#         collocates = np.unique(collocates, axis=0, return_counts=True)
#         #collocates = collocates[collocates[:,1].argsort()]
        
#         #print(collocates)
#         logdices = []

#         for row in range(len(collocates[0])):
#             collocate = collocates[0][row]
#             collocate_count = collocates[1][row]
#             if constrain_to == False:
#                 pass
#             else:
#                 if loaded_corpora[corpus_name]['vocab'][collocate] in constrain_to:
#                     pass
#                     #print(corpus['vocab'][collocate],' in constrain_to', constrain_to)
#                 else:
#                     continue
#             if collocate in coll_token_sequence:
#                 #print('match china')
#                 pass
#             elif collocate_count > 1:
#                 #14 + log2D=14+log2(2fxy/(fx+fy))

#                 if re.search(_RE_PUNCT,loaded_corpora[corpus_name]['vocab'][collocate]) is None: # FIX - HAVE OPION TO REMOVE PUNC
#                     if collocate_count >= cutoff:
#                         logdice = 14 + math.log2((2 * collocate_count) / (node_frequency + loaded_corpora[corpus_name]['frequency_lookup'][collocate]))
#                         mi = math.log2((loaded_corpora[corpus_name]['token_count'] * collocate_count) / (node_frequency * loaded_corpora[corpus_name]['frequency_lookup'][collocate]))
#                         logdices.append([logdice, mi, collocate, loaded_corpora[corpus_name]['vocab'][collocate], collocate_count, loaded_corpora[corpus_name]['frequency_lookup'][collocate]])
#                         #if logdice > 8:
#                         #print(collocate, node_frequency, loaded_corpora[corpus_name]['vocab'][collocate], collocates[1][row],logdice, mi)
#                         matches = np.where(collocates == collocate)[0]
#                         collocate_count = len(matches)
#         #        if (collocate_count > 5):
    
#     top_collocates = []
#     if stat == 'mi':
#         sorted_collocates = sorted(logdices, reverse=True, key=lambda x: x[1])[0:limit]
#         for row in sorted_collocates:
#             nodes.append((row[2], {'label': row[3], 'size': 1}))
#             edges.append((token_id, row[2], {'weight': 1}))
#             #print(row)
#             top_collocates.append(row[3])
#     else:
#         sorted_collocates = sorted(logdices, reverse=True, key=lambda x: x[0])[0:limit]
#         for row in sorted_collocates:
#             nodes.append((row[2], {'label': row[3], 'size': 1}))
#             edges.append((token_id, row[2], {'weight': row[0]}))
#             top_collocates.append(row[3])
    
#     if constrain_to == False:
#         for top_collocate in top_collocates:
#             top_collocate_elements, top_collocate_sorted, df = profile_prepare_collocates(corpus_name, top_collocate, top_collocate, constrain_to = top_collocates, context_length=context_length, limit = limit, stat=stat)
#             for edge in top_collocate_elements['edges']:
#                 edges.append(edge) 
    
    
#     #display(sorted(logdices, reverse=True, key=lambda x: x[0])[0:limit])
    
#     elements['nodes'] = nodes
#     elements['edges'] = edges
#     df = pd.DataFrame(sorted_collocates, columns = ['logdice', 'mi', 'collocate_token_id', 'collocate', 'collocate_count', 'collocate_token_frequency'])
#     # if output != False:
#     #     if output == 'file':
#     #         with open(output_dir + corpus_name + '/collocates-' + stat + '-' + nice_word + '.html', 'w', encoding='utf8') as f:
#     #             f.write(df.to_html(classes='table table-stripped'))
#     return elements, sorted_collocates, df


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()